# REINFORCE in lasagne (3 pts)

Just like we did before for q-learning, this time we'll design a lasagne network to learn `CartPole-v0` via policy gradient (REINFORCE).

Most of the code in this notebook is taken from Seminar4.0, so you'll find it more or less familiar and even simpler.

__Frameworks__ - we'll accept this homework in any deep learning framework. For example, it translates to TensorFlow almost line-to-line. However, we recommend you to stick to theano/lasagne unless you're certain about your skills in the framework of your choice.

In [1]:
%env THEANO_FLAGS='floatX=float32'
# import os
# if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
#     !bash ../xvfb start
#     %env DISPLAY=:1

env: THEANO_FLAGS='floatX=float32'


In [3]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

# plt.imshow(env.render("rgb_array"))

[2017-04-02 18:46:03,565] Making new env: CartPole-v0


# Building the network for REINFORCE

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states.

In [4]:
import theano
import theano.tensor as T

#create input variables. We'll support multiple states at once

states = T.matrix("states[batch,units]")
actions = T.ivector("action_ids[batch]")
cumulative_rewards = T.vector("R[batch] = r + gamma*r' + gamma^2*r'' + ...")

In [42]:
import lasagne
from lasagne.layers import *

#input layer
l_states = InputLayer((None,)+state_dim,input_var=states)

nn = lasagne.layers.DenseLayer(l_states, 128, nonlinearity=lasagne.nonlinearities.elu)
nn = lasagne.layers.DenseLayer(nn, 128,  nonlinearity=lasagne.nonlinearities.elu)
# <Your architecture. Please start with a 1-2 layers with 50-200 neurons>

#output layer
#this time we need to predict action probabilities,
#so make sure your nonlinearity forces p>0 and sum_p = 1
l_action_probas = DenseLayer(nn, num_units=n_actions,
                             nonlinearity=lasagne.nonlinearities.softmax)

#### Predict function

In [43]:
#get probabilities of actions
predicted_probas = get_output(l_action_probas)

#predict action probability given state
#if you use float32, set allow_input_downcast=True
predict_proba = theano.function([states], predicted_probas, allow_input_downcast=True)


#### Loss function and updates

We now need to define objective and update over policy gradient.

The objective function can be defined thusly:

$$ J \approx \sum  _i log \pi_\theta (a_i | s_i) \cdot R(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.


In [44]:
#select probabilities for chosen actions, pi(a_i|s_i)
predicted_probas_for_actions = predicted_probas[T.arange(actions.shape[0]),actions]

In [45]:
#REINFORCE objective function
J = T.mean(T.log(predicted_probas_for_actions) * cumulative_rewards)


In [46]:
#all network weights
all_weights = get_all_params(l_action_probas)

#weight updates. maximize J = minimize -J
updates = lasagne.updates.sgd(-J,all_weights,learning_rate=0.01)

In [47]:
train_step = theano.function([states,actions,cumulative_rewards],updates=updates, allow_input_downcast=True)

### Computing cumulative rewards

In [48]:
def get_cumulative_rewards(rewards, #rewards at each step
                           gamma = 0.99 #discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently
    
    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    
    R_ = [rewards[-1]]
    for r_t in rewards[-2::-1]:
        R_.append(r_t + gamma * R_[-1])
        
    return R_[::-1]
    
    
    

In [38]:
assert len(get_cumulative_rewards(range(100))) == 100
assert np.allclose(get_cumulative_rewards([0,0,1,0,0,1,0],gamma=0.9),[1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,-2,3,-4,0],gamma=0.5), [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,2,3,4,0],gamma=0), [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


### Playing the game

In [39]:
def generate_session(t_max=1000):
    """play env with REINFORCE agent and train at the session end"""
    
    #arrays to record session
    states,actions,rewards = [],[],[]
    
    s = env.reset()
    
    for t in range(t_max):
        
        #action probabilities array aka pi(a|s)
        action_probas = predict_proba([s])[0] 
        
        a = np.random.choice(np.arange(n_actions), p=action_probas)
        
        new_s,r,done,info = env.step(a)
        
        #record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)
        
        s = new_s
        if done: break
            
    cumulative_rewards = get_cumulative_rewards(rewards)
    train_step(states,actions,cumulative_rewards)
            
    return sum(rewards)
        

In [51]:
for i in range(100):
    
    rewards = [generate_session() for _ in range(100)] #generate new sessions
    
    print ("mean reward:%.3f"%(np.mean(rewards)))

    if np.mean(rewards) > 300:
        print ("You Win!")
        break

mean reward:34.270
mean reward:40.130
mean reward:61.260
mean reward:57.890
mean reward:72.120
mean reward:37.020
mean reward:139.670
mean reward:157.230
mean reward:88.450
mean reward:84.900
mean reward:132.200
mean reward:160.010
mean reward:155.120
mean reward:175.470
mean reward:194.290
mean reward:223.820
mean reward:193.460
mean reward:311.150
You Win!


### Video

In [53]:
env.close()

[2017-04-02 19:02:55,910] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/ubuntu/videos')


In [ ]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()


In [ ]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices